In [1]:
import pandas as pd 

In [2]:
produce_data = pd.read_csv('/Users/ajarbuckle/Desktop/Produce Project/Original Data/produce_price_timeseries.csv')

# Getting rid of unnecessary columns 
produce_data = produce_data[['Commodity', 'Date', 'Average']]

# Changing some column names 
produce_data.rename(columns={'Average': 'Price', 'Commodity': 'Item'}, inplace=True)

produce_data

,Item,Date,Price
0,Tomato Big(Nepali),2013-06-16,37.5
1,Tomato Small(Local),2013-06-16,29.0
2,Potato Red,2013-06-16,20.5
3,Potato White,2013-06-16,15.5
4,Onion Dry (Indian),2013-06-16,29.0
...,...,...,...
197156,Garlic Dry Nepali,2021-05-13,110.0
197157,Fish Fresh(Rahu),2021-05-13,275.0
197158,Fish Fresh(Bachuwa),2021-05-13,230.0
197159,Fish Fresh(Chhadi),2021-05-13,225.0


In [3]:
# Checking to see how many commodities are in the dataframe so that I can see how many are filtered out in each step 
# of my criteria 

produce_data['Item'].nunique()

132

In [4]:
# Making sure that the 'Date' column is a date variable

produce_data['Date'] = pd.to_datetime(produce_data['Date'])
produce_data.dtypes

Item             object
Date     datetime64[ns]
Price           float64
dtype: object

In [5]:
# Creating a new dataframe that has the first and last price recording for each item, plus how many days they span ('Total_Span'), 
# plus how many days would be 75% of the total days spanned ('Min_Days')

# I will use this dataframe to determine which items to do time series forecasting on 

forecast_items = produce_data.groupby('Item')['Date'].agg(['min', 'max'])

# Renaming the 'min' and 'max' columns for clarity 
forecast_items.rename(columns={'min': 'First_Day', 'max':'Last_Day'}, inplace=True)

forecast_items['Total_Span'] = (forecast_items['Last_Day'] - forecast_items['First_Day'] + pd.Timedelta(days=1)).dt.days.astype(int)
forecast_items['Min_Days'] = (forecast_items['Total_Span'] * 0.75).astype(int)

forecast_items

,First_Day,Last_Day,Total_Span,Min_Days
Item,,,,
Apple(Fuji),2019-05-31,2021-05-13,714,535
Apple(Jholey),2013-06-16,2021-05-12,2888,2166
Arum,2013-06-25,2021-05-13,2880,2160
Asparagus,2013-06-16,2021-05-13,2889,2166
Bakula,2013-09-29,2021-04-27,2768,2076
...,...,...,...,...
Turnip,2013-06-16,2021-04-27,2873,2154
Turnip A,2013-06-16,2021-05-13,2889,2166
Water Melon(Dotted),2019-06-04,2021-04-29,696,522


In [6]:
# Creating a new column in the forecast_items dataframe shows how many days each item had its price recorded 

# Get the number of days each item was recorded
item_counts = produce_data.groupby('Item').size().reset_index(name='Total_Days')

# Merge that information onto the forecast_items dataframe
forecast_items = pd.merge(forecast_items, item_counts, on='Item', how='left')

forecast_items

,Item,First_Day,Last_Day,Total_Span,Min_Days,Total_Days
0,Apple(Fuji),2019-05-31,2021-05-13,714,535,395
1,Apple(Jholey),2013-06-16,2021-05-12,2888,2166,2683
2,Arum,2013-06-25,2021-05-13,2880,2160,2020
3,Asparagus,2013-06-16,2021-05-13,2889,2166,1055
4,Bakula,2013-09-29,2021-04-27,2768,2076,1131
...,...,...,...,...,...,...
127,Turnip,2013-06-16,2021-04-27,2873,2154,420
128,Turnip A,2013-06-16,2021-05-13,2889,2166,2538
129,Water Melon(Dotted),2019-06-04,2021-04-29,696,522,42
130,Water Melon(Green),2013-06-16,2021-05-13,2889,2166,2679


In [7]:
# Now getting rid of any commodities whose prices were not recorded on at least 75% of the days that their first and last days of recording span

forecast_items = forecast_items[forecast_items['Total_Days'] >= forecast_items['Min_Days']]
forecast_items 

,Item,First_Day,Last_Day,Total_Span,Min_Days,Total_Days
1,Apple(Jholey),2013-06-16,2021-05-12,2888,2166,2683
5,Bamboo Shoot,2013-06-16,2021-05-13,2889,2166,2744
6,Banana,2013-06-16,2021-05-13,2889,2166,2744
9,Bitter Gourd,2013-06-16,2021-05-13,2889,2166,2670
10,Bottle Gourd,2013-06-16,2021-05-13,2889,2166,2700
...,...,...,...,...,...,...
122,Tomato Big(Nepali),2013-06-16,2021-05-13,2889,2166,2507
124,Tomato Small(Local),2013-06-16,2021-05-13,2889,2166,2741
126,Tomato Small(Tunnel),2019-05-31,2021-05-13,714,535,686
128,Turnip A,2013-06-16,2021-05-13,2889,2166,2538


In [8]:
# Now getting rid of any commodities that don't have data for at least 4 years

forecast_items = forecast_items[forecast_items['Total_Span'] >= 365 * 4]
forecast_items

,Item,First_Day,Last_Day,Total_Span,Min_Days,Total_Days
1,Apple(Jholey),2013-06-16,2021-05-12,2888,2166,2683
5,Bamboo Shoot,2013-06-16,2021-05-13,2889,2166,2744
6,Banana,2013-06-16,2021-05-13,2889,2166,2744
9,Bitter Gourd,2013-06-16,2021-05-13,2889,2166,2670
10,Bottle Gourd,2013-06-16,2021-05-13,2889,2166,2700
11,Brd Leaf Mustard,2013-06-16,2021-05-13,2889,2166,2742
12,Brinjal Long,2013-06-16,2021-05-13,2889,2166,2736
16,Cabbage(Local),2013-06-16,2021-05-13,2889,2166,2749
18,Capsicum,2013-06-16,2021-05-13,2889,2166,2725
19,Carrot(Local),2013-06-16,2021-05-13,2889,2166,2736


In [9]:
len(forecast_items)

55

In [10]:
# Now I'm going to read in another CSV that has quantiative information on seasonality

seasonality_data = pd.read_csv('/Users/ajarbuckle/Desktop/Produce Project/Derived Tables/produce_seasonality.csv')

seasonality_data

,item,low_price_month,low_price_season,low_price,peak_price_month,peak_price_season,peak_price,seasonal_variability,seasonal_variability_rank
0,Maize,June,Summer,5.50,January,Winter,72.50,13.18,1
1,Pointed Gourd(Local),June,Summer,35.94,February,Winter,213.60,5.94,2
2,Guava,September,Fall,36.56,March,Spring,195.71,5.35,3
3,Carrot(Terai),May,Spring,17.88,November,Fall,86.67,4.85,4
4,Kinnow,May,Spring,50.62,November,Fall,230.00,4.54,5
...,...,...,...,...,...,...,...,...,...
127,Fish Fresh,July,Summer,227.40,October,Fall,241.68,1.06,128
128,Tofu,July,Summer,78.40,September,Fall,81.56,1.04,129
129,Mango(Calcutte),May,Spring,133.08,April,Spring,135.00,1.01,130
130,Litchi(Indian),June,Summer,142.62,June,Summer,142.62,1.00,131


In [11]:
# Renaming the 'item' column to 'Item' for simplicity's sake when joining
seasonality_data.rename(columns={'item': 'Item'}, inplace=True)

# Eliminating unnecessary columns 
seasonality_data = seasonality_data[['Item', 'seasonal_variability']]

seasonality_data

,Item,seasonal_variability
0,Maize,13.18
1,Pointed Gourd(Local),5.94
2,Guava,5.35
3,Carrot(Terai),4.85
4,Kinnow,4.54
...,...,...
127,Fish Fresh,1.06
128,Tofu,1.04
129,Mango(Calcutte),1.01
130,Litchi(Indian),1.00


In [12]:
# Now joining the seasonal_variability dataframe onto the forecast_items dataframe so that I can whittle it down even further to the top 
# 5 commodities with the most seasonal variability 

forecast_items = pd.merge(forecast_items, seasonality_data, on='Item', how='left')
forecast_items

,Item,First_Day,Last_Day,Total_Span,Min_Days,Total_Days,seasonal_variability
0,Apple(Jholey),2013-06-16,2021-05-12,2888,2166,2683,1.20
1,Bamboo Shoot,2013-06-16,2021-05-13,2889,2166,2744,1.20
2,Banana,2013-06-16,2021-05-13,2889,2166,2744,1.35
3,Bitter Gourd,2013-06-16,2021-05-13,2889,2166,2670,4.01
4,Bottle Gourd,2013-06-16,2021-05-13,2889,2166,2700,1.42
5,Brd Leaf Mustard,2013-06-16,2021-05-13,2889,2166,2742,2.17
6,Brinjal Long,2013-06-16,2021-05-13,2889,2166,2736,1.61
7,Cabbage(Local),2013-06-16,2021-05-13,2889,2166,2749,2.49
8,Capsicum,2013-06-16,2021-05-13,2889,2166,2725,2.34
9,Carrot(Local),2013-06-16,2021-05-13,2889,2166,2736,3.37


In [13]:
forecast_items.sort_values('seasonal_variability', ascending = False)

,Item,First_Day,Last_Day,Total_Span,Min_Days,Total_Days,seasonal_variability
39,Pointed Gourd(Local),2013-06-16,2021-05-13,2889,2166,2260,5.94
15,Coriander Green,2013-06-16,2021-05-13,2889,2166,2741,4.16
3,Bitter Gourd,2013-06-16,2021-05-13,2889,2166,2670,4.01
9,Carrot(Local),2013-06-16,2021-05-13,2889,2166,2736,3.37
14,Christophine,2013-06-16,2021-04-27,2873,2154,2225,3.18
10,Cauli Local,2013-06-16,2021-05-13,2889,2166,2750,2.88
33,Okara,2013-06-16,2021-05-13,2889,2166,2663,2.79
34,Onion Dry (Indian),2013-06-16,2021-05-13,2889,2166,2742,2.64
16,Cow pea(Long),2013-06-16,2021-05-13,2889,2166,2251,2.64
7,Cabbage(Local),2013-06-16,2021-05-13,2889,2166,2749,2.49


In [14]:
# Filtering the forecast_items dataframe so it only contains the commodities with the top 5 highest seasonal variability 

forecast_items = forecast_items.nlargest(5, 'seasonal_variability')
forecast_items.sort_values('seasonal_variability', ascending = False)

,Item,First_Day,Last_Day,Total_Span,Min_Days,Total_Days,seasonal_variability
39,Pointed Gourd(Local),2013-06-16,2021-05-13,2889,2166,2260,5.94
15,Coriander Green,2013-06-16,2021-05-13,2889,2166,2741,4.16
3,Bitter Gourd,2013-06-16,2021-05-13,2889,2166,2670,4.01
9,Carrot(Local),2013-06-16,2021-05-13,2889,2166,2736,3.37
14,Christophine,2013-06-16,2021-04-27,2873,2154,2225,3.18


In [ ]:
# Now I know which items are worth building time series forecast model for